In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

import datetime

import hydrofunctions as hf

import fbprophet 

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim
import geopy.distance

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [2]:
## Reading in site data from 'Gage_sites.txt'
file = 'Gage_sites.txt'
sites = pd.read_csv(file, sep = '\t', header = 39, skiprows = [40], dtype = {"site_no" : "str"})

sites.drop([65, 71, 77, 117, 118, 138, 203, 225, 320, 330, 331, 338], inplace = True) ##Removing sites without data

site_no = list(sites["site_no"])
site_nm = list(sites["station_nm"])
site_loc = sites.filter(['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va'])

In [3]:
dbname = 'usgs_stream_db'
username = 'cadeadams'

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

if not database_exists(engine.url):
    create_database(engine.url)

In [20]:
con = None
con = psycopg2.connect(database = dbname, user = username)

sql_query = """
SELECT * FROM site_locations;
"""

query_results = pd.read_sql_query(sql_query,con)

location = [40.368855, -104.5372]

site_no = query_results["site_no"]
site_lat = query_results["dec_lat_va"]
site_long = query_results["dec_long_va"]

sites_coord = pd.DataFrame([site_no, site_lat, site_long])
sites_coord = sites_coord.T

distance = []
for i in range(len(sites_coord)) :
    distance.append(geopy.distance.distance(location, sites_coord.iloc[i,1:]).miles)

query_results["distance"] = distance
query_results = query_results.sort_values(by = ["distance"])
loc1_lat = float(query_results[0:1]["dec_lat_va"])
loc1_lon = float(query_results[0:1]["dec_long_va"])
location_1 = pd.DataFrame([loc1_lat, loc1_lon])
print(query_results[1:2]["station_nm"].iloc[0])
print(loc1_lat)

#site_data_from_sql = pd.read_sql_query(sql_query_dat,con)
#site_model_from_sql = pd.read_sql_query(sql_query_mod,con)

BIG DRY CREEK AT MOUTH NEAR FORT LUPTON, CO
40.11609435


# Everything below here is for collection and placing into PostgreSQL database. FBProphet models are also performed over all data collected.

In [4]:
#site_no.index('394220106431500') ## Has water temp data. Need to extract...

site_loc.to_sql('site_locations', engine, if_exists='replace')

In [5]:
##
# Pulling in data using hydrofunctions and saving to PostgreSQL database.
##
start = '2000-01-01'
end = str(datetime.datetime.today().strftime('%Y-%m-%d')) #Gets today's date.

for site in site_no :
    herring = hf.NWIS(site, 'dv', start, end)
    herring.get_data()
    my_dict = herring.json()
    df = hf.extract_nwis_df(my_dict)
    df.rename(index=str, columns = {"USGS:"+site+":00060:00003" : "y", 
                                    "USGS:"+site+":00060:00003_qualifiers" : "flags"}, 
             inplace = True)
    df['ds'] = df.index[:]
#    df['ds'].str.split(pat = ' ', expand = True)
    df.to_sql("n"+str(site), engine, if_exists='replace')

In [17]:
##
# Modeling data using fbprophet and saving to PostgreSQL database.
##
con = None
con = psycopg2.connect(database = dbname, user = username)

for gage in site_no :
    sql_query = """
    SELECT * FROM n"""+gage+""";
    """
    site_data_from_sql = pd.read_sql_query(sql_query,con)
    df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
    df_prophet.fit(site_data_from_sql)
    df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
    df_forecast = df_prophet.predict(df_forecast)
    df_forecast.to_sql("n"+str(gage)+"_forecast", engine, if_exists='replace')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

In [16]:
df.plot()
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.title('Daily Mean Discharge for Roaring Fork River')
plt.savefig('RRFork1.png')
plt.close()

In [9]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
df_prophet.fit(df)

In [10]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

In [12]:
df_prophet.plot_components(df_forecast)
plt.savefig('RRFork1_Components.png')
plt.close()

In [18]:
site_model_from_sql.tail()

,index,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
7691,7691,2021-01-21,0.967861,-0.579543,1.447869,0.967861,0.967861,-0.477573,-0.477573,-0.477573,...,0.020013,0.020013,0.020013,-2.623681,-2.623681,-2.623681,0.0,0.0,0.0,0.490288
7692,7692,2021-01-22,0.968049,-0.603034,1.530899,0.968049,0.968049,-0.491970,-0.491970,-0.491970,...,0.016397,0.016397,0.016397,-2.634462,-2.634462,-2.634462,0.0,0.0,0.0,0.476079
7693,7693,2021-01-23,0.968238,-0.522904,1.572616,0.968238,0.968238,-0.525330,-0.525330,-0.525330,...,-0.008017,-0.008017,-0.008017,-2.643407,-2.643407,-2.643407,0.0,0.0,0.0,0.442908
7694,7694,2021-01-24,0.968426,-0.578047,1.476846,0.968426,0.968426,-0.533844,-0.533844,-0.533844,...,-0.009711,-0.009711,-0.009711,-2.650228,-2.650228,-2.650228,0.0,0.0,0.0,0.434582
7695,7695,2021-01-25,0.968615,-0.388971,1.673874,0.968615,0.968615,-0.530558,-0.530558,-0.530558,...,-0.001953,-0.001953,-0.001953,-2.654699,-2.654699,-2.654699,0.0,0.0,0.0,0.438057
